In [ ]:
from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi
import os
import gridfs
from dotenv import load_dotenv


load_dotenv()
uri = os.getenv("URL")
client = MongoClient(uri, server_api=ServerApi('1'))

try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

    
db = client['Lecture']
fs = gridfs.GridFS(db)

def upload_files(directory):
    for foldername, subfolders, filenames in os.walk(directory):
        for filename in filenames:
            file_path = os.path.join(foldername, filename)
            
            with open(file_path, 'rb') as file:
                relative_path = os.path.relpath(file_path, directory)
                file_id = fs.put(file, filename=relative_path, metadata={
                    'path': relative_path,
                    'folder': foldername,
                    'original_name': filename
                })
                print(f'File uploaded: {relative_path} with ID: {file_id}')

directory_path = os.getenv("data_folder")
upload_files(directory_path)
